In [1]:
from utils import prepare_jupyter
prepare_jupyter()

In [ ]:
import numpy as np
import networkx as nx

from IPython.display import Image
from structure import RandomForestEnsemble
from data import Dataset

train_set, val_set = Dataset.create_iris().split(0.2)

params = {
    'n_estimators': 5,
    'max_depth': 2
}

rf = RandomForestEnsemble(params)
rf.fit(train_set)

tree_a, tree_b = rf.trees[0], rf.trees[4]

/Users/tomek/.virtualenvs/masters/lib/python3.6/site-packages/lightgbm/__init__.py:47: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [ ]:
from itertools import product

# nx_a = tree_a.tree
# nx_b = tree_b.tree

def compare_nodes(node_a, node_b):
    if node_a['is_leaf'] and node_b['is_leaf']:
        return node_a['target'] == node_b['target']
    elif not node_a['is_leaf'] and not node_b['is_leaf']:
        return node_a['threshold'] == node_b['threshold'] and node_a['feature'] == node_b['feature']
    
    return False

results = []
for tree_a, tree_b in product(rf.trees, rf.trees):
    if tree_a == tree_b:
        continue
        
    result = nx.graph_edit_distance(tree_a.tree, tree_b.tree, node_match=compare_nodes)
    results.append(results)
    
results = np.array(results)
# nx.graph_edit_distance(nx_a, nx_b, node_match=compare_nodes)

In [ ]:
def same_nodes_similarity(tree_a, tree_b):
    # TODO
    pass